# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.

In [2]:
import pandas as pd
from scipy.stats import shapiro

In [3]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

Объединим 2 массива в датафрейм:

In [4]:
df = pd.DataFrame({'petersburg': pd.Series(petersburg), 'magadan': pd.Series(magadan)})
df

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


Заполняем отсутствующее значение столбца "petersburg" средним значением:

In [5]:
df['petersburg'] = df['petersburg'].fillna(df['petersburg'].mean())
df

,petersburg,magadan
0,0.097400,0.1033
1,0.135200,0.0915
2,0.081700,0.0781
3,0.101600,0.0685
4,0.096800,0.0677
5,0.106400,0.0697
6,0.105000,0.0764
7,0.103443,0.0689


_________________________

В качестве **теста на нормальность** используем тест Шапиро-Уилка. Данный выбор связан с небольшим размером выборки данных (тест Д'Агостино используется для выборки от 20 значений и более).

In [6]:
# определим нулевую и альтернативную гипотезы:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

In [7]:
# Установим уровень значимости:
alpha = 0.05

In [8]:
_, p = shapiro(df['petersburg']) # отбросим путстое значение столбца при выполнении теста
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.123
Данные распределены нормально


In [9]:
## Тест Шапиро-Уилка
_, p = shapiro(df['magadan'])
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.036
Данные не распределены нормально (мы отвергаем H0)


Таким образом по результатам теста Шапиро-Уилка данные столбца "petersburg" распределены нормально, а данные столбца "magadan" не соответствуют нормальному распределению.

______________________________________

В качестве **теста на наличие корреляции** (т.к. имеем дело с количественными переменными) используем тест "кореляция Спирмена".

In [10]:
round(df.corr(method = 'spearman'), 2)

,petersburg,magadan
petersburg,1.00,0.17
magadan,0.17,1.00


Результаты теста на корреляцию Спирмена свидетельствуют об отсутствии связи (либо очень слабой связи) между признаками (коэф.корреляции < 0.3)

Считаю, что использовать в данном случае корреляционные тесты не корректно, т.к. в данном случае нет как таковых пар значений (данные в выборках не связаны). Целесообразно применять только тест на сравнение выборок.

_________________________________________

В качестве **теста на сравнение выборок** (с учетом ненормальности распределения столбца "magadan") используем непараметрический тест - сумма рангов Уилкоксона (группы происходят из разных совокупностей).  

In [11]:
# переопределим нулевую и альтернативную гиротезу в части разницы размеров мидий:
H0 = 'Нет значимой разницы в размере мидий.'
Ha = 'Есть значимая разница в размере мидий.'

In [12]:
# определим статистическую значимость для теста "Сумма рангов Уилкоксона":
from scipy.stats import ranksums
_, p = ranksums(df['petersburg'], df['magadan'])
# Интерпретация 
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.004574439456210679 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница в размере мидий.


Т.к. *p-value* нашего теста ниже уровня значимости, это означает, что ваши результаты статистически значимы и согласуются с альтернативной гипотезой. Таким образом существует **значимая разница** в размере мидий.

Определим  также фактическую разницу среднего значения размера мидий в зависимоти от города происхождения (раз уж так сформулирован вопрос в задании):

In [14]:
print(df['petersburg'].mean() - df['magadan'].mean())

0.025430357142857143


Разница средних значений размера мидий составила 2.54 см